In [6]:
with open("q16_input.txt", "r") as fp:
    f_lines = fp.readlines()

In [7]:
input_str = f_lines[0].strip()

In [ ]:
input_str

In [ ]:
conv_map = {
    "0" : "0000",
    "1" : "0001",
    "2" : "0010",
    "3" : "0011",
    "4" : "0100",
    "5" : "0101",
    "6" : "0110",
    "7" : "0111",
    "8" : "1000",
    "9" : "1001",
    "A" : "1010",
    "B" : "1011",
    "C" : "1100",
    "D" : "1101",
    "E" : "1110",
    "F" : "1111",
}


In [ ]:
conv_str = ""
for c in input_str:
    conv_str += conv_map[c]

In [ ]:
conv_str = "00111000000000000110111101000101001010010001001000000000"

In [ ]:
mask = "VVVTTTILLLLLLLLLLLLLLLAAAAAAAAAAABBBBBBBBBBBBBBBB"

In [ ]:
from collections import deque

In [ ]:
def parse_packet(i_start, i_end):
    packet = conv_str[i_start:i_end]
    stack = []
    i = 0
    actions = deque([(3, "v"), (3, "id")])
    while i < len(conv_str):
        step, step_type = actions.popleft()
        output = conv_str[i: i + step]
        decode_output = int(output, 2)
        stack.append((decode_output, step_type))
        if step_type == "id":
            if decode_output == 4:
                # literal value
                pass
            else:
                actions.append((1, "l"))
        elif step_type == "l":
            if decode_output == 0:
                actions.append((15, "n"))
            elif decode_output == 1:
                actions.append((11, "n"))
        elif step_type == "n":
            for _ in range(stack[-1][0]):
                print(step)
                actions.append((step, "number"))
    return stack

In [ ]:
stack = []
i = 0
actions = deque([(3, "v"), (3, "id")])
while i < len(conv_str):
    step, step_type = actions.popleft()
    output = conv_str[i: i + step]
    decode_output = int(output, 2)
    stack.append((decode_output, step_type))
    if step_type == "id":
        if decode_output == 4:
            # literal value
            pass
        else:
            actions.append((1, "l"))
    elif step_type == "l":
        if decode_output == 0:
            actions.append((15, "n"))
        elif decode_output == 1:
            actions.append((11, "n"))
    elif step_type == "n":
        for _ in range(stack[-1][0]):
            print(step)
            actions.append((step, "number"))
    print(mask[i: i + step], conv_str[i: i + step])
    i = i + step

In [ ]:
stack

# I GIVE UP after an hour
# CHEATED: https://github.com/mebeim/aoc/blob/master/2021/solutions/day16.py

In [11]:
class Bitstream:
	def __init__(self, file):
		rawdata = bytes.fromhex(file)
		self.bits = ''.join(map('{:08b}'.format, rawdata))
		self.pos = 0

	def decode_int(self, nbits):
		res = int(self.bits[self.pos:self.pos + nbits], 2)
		self.pos += nbits
		return res

	def decode_n_packets(self, n):
		return [self.decode_one_packet() for _ in range(n)]

	def decode_len_packets(self, length):
		end = self.pos + length
		pkts = []

		while self.pos < end:
			pkts.append(self.decode_one_packet())

		return pkts

	def decode_value_data(self):
		value = 0
		group = 0b10000

		while group & 0b10000:
			group = self.decode_int(5)
			value = (value << 4) + (group & 0b1111)

		return value

	def decode_operator_data(self):
		if self.decode_int(1):
			return self.decode_n_packets(self.decode_int(11))
		return self.decode_len_packets(self.decode_int(15))

	def decode_packet_data(self, tid):
		if tid == 4:
			return self.decode_value_data()
		return self.decode_operator_data()

	def decode_one_packet(self):
		version = self.decode_int(3)
		tid     = self.decode_int(3)
		data    = self.decode_packet_data(tid)
		return (version, tid, data)

def sum_versions(packet):
	v, tid, data = packet
	return v if tid == 4 else v + sum(map(sum_versions, data))

from math import prod
def evaluate(packet):
	_, tid, data = packet

	if tid == 4:
		return data

	values = map(evaluate, data)

	if tid == 0: return sum(values)
	if tid == 1: return prod(values)
	if tid == 2: return min(values)
	if tid == 3: return max(values)

	a, b = values

	if tid == 5: return int(a > b)
	if tid == 6: return int(a < b)
	if tid == 7: return int(a == b)

	raise NotImplementedError('Unimplemented tid={}'.format(tid))


In [12]:
stream = Bitstream(input_str)

In [13]:
packet = stream.decode_one_packet()
vsum   = sum_versions(packet)
result = evaluate(packet)

print(vsum)
print(result)

877
194435634456
